In [1]:
from connection import ConnectionTask
import time
c=ConnectionTask()

In [94]:
# 流量:补偿卡在充值中的订单
order_list=list(c.pmdb.order_liuliang.find({'displayStatus':{'$in':['充值中']},
                                          'updatedAt':{'$gte':(time.time()-3600*24*21)*1000,
                                                      '$lt':(time.time()-3600*3)*1000},
                                         },
                                         {'id':1,'partnerId':1,'orderType':1,'_id':0},sort=[('_id',-1)]))
for item in order_list:
    item['orderId']=item.pop('id')
    print(item)
#     continue
    message={
        "taskName":"worker.dhst.dhst_fetch",
        "queue":"dhst_tasks",
        "args":item
    }
    c.prdb_mast(0).rpush('session_task_hub',message)

{'orderType': 'liuliang', 'partnerId': 'ala202918673458893', 'orderId': 'ala202918673458893'}
{'orderType': 'liuliang', 'partnerId': 'ala202903553093369', 'orderId': 'ala202903553093369'}


In [16]:
start = time.mktime((2017,12,27,0,0,0,0,0,0))*1000
end = time.mktime((2017,12,27,24,0,0,0,0,0))*1000
order_id_list=[]
for item in c.pmdb.order_waimai.find({'status':4,'createdAt':{'$gte':start,'$lt':end}}):
    order_id_list.append(item['id'])

In [ ]:
start = time.mktime((2018,1,4,0,0,0,0,0,0))*1000
end = time.mktime((2018,1,4,24,0,0,0,0,0))*1000
c.pmdb.order_waimai.find({'status':6,'createdAt':{'$gte':start,'$lt':end}})

In [8]:
order_id_list="""tcb202915807156171
bp203057003661345
ala203057141057535
bp202896368247630
bp202915679999379""".split('\n')

In [9]:
order_id_list

['tcb202915807156171',
 'bp203057003661345',
 'ala203057141057535',
 'bp202896368247630',
 'bp202915679999379']

In [10]:
# 外卖:补偿特定的订单
order_list=c.pmdb.order_waimai.find({'id':{'$in':order_id_list}},
                                    {'id':1,'partnerId':1,'_id':0},sort=[('_id',-1)])

for item in order_list:
    item['orderId']=item.pop('id')
    message={
        "taskName":"worker.ele.order_status_polling",
        "queue":"ele_tasks",
        "args":item
    }
    print(item)
    c.prdb_mast(0).rpush('session_task_hub',message)
#     time.sleep(0.5)

{'partnerId': '1215423028577073357', 'orderId': 'ala203057141057535'}
{'partnerId': '3017260545087127632', 'orderId': 'bp203057003661345'}
{'partnerId': '1214848111416627404', 'orderId': 'tcb202915807156171'}
{'partnerId': '3016636229697312946', 'orderId': 'bp202915679999379'}
{'partnerId': '3016567212509553825', 'orderId': 'bp202896368247630'}


In [16]:
# 外卖:补偿卡在等待配送中的订单
order_list=list(c.pmdb.order_waimai.find({'displayStatus':{'$in':['配送中','等待商家接单','美食制作中']},
                                          'updatedAt':{'$gte':(time.time()-3600*24)*1000,
                                                      '$lt':(time.time()-360)*1000},
                                         },
                                         {'id':1,'partnerId':1,'_id':0},sort=[('_id',-1)]))
for item in order_list:
    item['orderId']=item.pop('id')
# pprint(order_list)

for item in order_list:
    message={
        "taskName":"worker.ele.order_status_polling",
#         "taskName":"worker.ele.ele_fetch",
        "queue":"ele_tasks",
        "args":item
    }
    print(item)
    c.prdb_mast(0).rpush('session_task_hub',message)

{'partnerId': '3019480715385034907', 'orderId': 'bp203556204888153'}
{'partnerId': '1216926293661332557', 'orderId': 'bp203556202283062'}
{'partnerId': '1216926259907681302', 'orderId': 'bp203556186931287'}
{'partnerId': '3019480638847537265', 'orderId': 'bp203556185997421'}
{'partnerId': '3019480544987490375', 'orderId': 'bp203556179066966'}
{'partnerId': '3019480581738684605', 'orderId': 'bp203556176953450'}
{'partnerId': '3019480580647473256', 'orderId': 'bp203556172759145'}
{'partnerId': '3019480529116272800', 'orderId': 'ala203556124786788'}
{'partnerId': '3019480492172641384', 'orderId': 'bp203556124688422'}
{'partnerId': '3019480528814857466', 'orderId': 'bp203556123885645'}
{'partnerId': '3019480452150604012', 'orderId': 'bp203556122968139'}
{'partnerId': '3019480562795504873', 'orderId': 'bp203556115890249'}
{'partnerId': '3019480525753089024', 'orderId': 'bp203556112597087'}
{'partnerId': '3019480484455685123', 'orderId': 'bp203556101455964'}
{'partnerId': '301948050850598205

In [32]:
# 猫眼:补偿卡在出票中的订单
order_list=list(c.pmdb.order_dianying.find({'displayStatus':{'$in':['出票中']},
                                          'updatedAt':{'$gte':(time.time()-3600*24*10)*1000,
                                                      '$lt':(time.time()-60*15)*1000},
                                         },
                                         {'id':1,'appCode':1,'channel':1,'partnerId':1,'_id':0},sort=[('_id',-1)]))

for item in order_list:
#     {'appCode': 'bestpay', 'partnerOrderId': 69806476, 'channel': 'maoyan', 'orderId': 'bp203313712148733'}
    item['orderId']=item.pop('id')
    item['partnerOrderId']=item.pop('partnerId')
    message={
        "taskName":"worker.maoyan.order_status_polling",
        "queue":"maoyan_tasks",
        "args":item
    }
    print(item)
    c.prdb_mast(0).rpush('session_task_hub',message)

In [153]:
# 外卖:补偿卡在等待商家接单的订单
for item in c.pmdb.order_waimai.find({'displayStatus':{'$in':[
#     '等待商家接单',
#     '商家已接单',
#     '美食制作中',
#     '已支付'
]},
                                          'updatedAt':{'$gte':(time.time()-3600*24*21)*1000,
                                                      '$lt':(time.time()-15*60*60)*1000},
                                         },
                                         {'id':1,'partnerId':1,'_id':0},sort=[('_id',-1)]):
    item['orderId']=item.pop('id')
    message={
#         "taskName":"worker.ele.order_status_polling",
        "taskName":"worker.ele.ele_fetch",
        "queue":"ele_tasks",
        "args":item
    }
    print(item)
    c.prdb_mast(0).rpush('session_task_hub',message)

{'partnerId': '1215147653433209085', 'orderId': 'ala202987426715740'}


In [156]:
# 补偿卡在待支付中的订单
count=0
for order_lite in c.pmdb.order_lite_list.find({'status':2,'createdAt':{'$gte':(time.time()-3600*24*25)*1000,
                                                                       '$lt':(time.time()-3600*24)*1000}},
                                              {'id':1,'partnerId':1,'_id':0,'userId':1,'appCode':1,'orderType':1},
                                             sort=[('_id',-1)]):
    order_message=c.pmdb[f'order_{order_lite["orderType"]}'].find_one({'id':order_lite['id']},
                                                                      {'id':1,'partnerId':1,'_id':0,'userId':1,'appCode':1,'orderType':1})
    if not order_message:
        continue
    order_message['orderId']=order_message.pop('id')
#     print(order_message)
    message={
        "taskName":"worker.expire.expire_fetch",
        "queue":"expire_tasks",
        "args":order_message,
    }
    count+=1
    print(f'>>>orderId:{order_message["orderId"]},Count:{count}<<<',end='\r')
    c.prdb_mast(0).rpush('session_task_hub',message)

In [9]:
order_list="""
ala203403526463556
bp203313511215249
src203313490739131
ddc203313475141553
bp203313472942199
bp203313436753381
bp203313420631514
bp203313394532224
bp203313357716922
bp203313344855904
bp203313324817820
bp203313312087883
bp203313300058119
ddc203313288117638
ala203313271586611
bp203313255742832
ala203313240653601
vjr203313237798885
bp203313230052698
bp203313194151877
bp203313171856699
smy203313167138102
bp203313154043826
smy203313144737707
smy203313131208421
bp203313127145186
bp203313121131794
bp203313114328986
bp203313108795090
ala203313100799689
bp203313085230976
bp203313076289208
bp203313059511983
vjr203313055055531
ddc203313034739344
bp203313028710022
bp203313023364945
bp203313010257741
bp203312992825152
bp203312989388400
bp203312984931944
bp203312979311768
bp203312978001047
ala203312976821398
bp203312973413522
bp203312970136716
bp203312960830594
bp203312960581401
ala203312955718776
ala203312951377474
bp203312951013136
bp203312939465835
bp203312938679402
bp203312928455780
bp203312922046190
bp203312917560863
bp203312917429790
bp203312902647515
ala203312898948620
bp203312889278153
bp203312885579264
bp203300475310354
bp203293483272894
bp203293482588582
bp203293427029838
bp203292942964652
bp203292933294251
bp203292905346948
bp203292818708223
bp203292818461969
bp203292346061997
bp203292345553611
bp203292323779730
bp203292315944434
bp203291566985305
bp203289666600334
bp203289662507552
bp203289571658547
bp203289569475805
bp203289555421553
bp203289553222836
bp203289552043182
bp203289550732459
smy203258539590802""".split("\n")

In [15]:
count=0
for order_lite in c.pmdb.order_lite_list.find({'id':{"$in":order_list}},
                                              {'id':1,'partnerId':1,'_id':0,'userId':1,'appCode':1,'orderType':1},
                                             sort=[('_id',-1)]):
    order_message=c.pmdb[f'order_{order_lite["orderType"]}'].find_one({'id':order_lite['id']},
                                                                      {'id':1,'partnerId':1,'_id':0,'userId':1,'appCode':1,'orderType':1})
    if not order_message:
        continue
    order_message['orderId']=order_message.pop('id')
#     print(order_message)
    message={
        "taskName":"worker.expire.expire_fetch",
        "queue":"expire_tasks",
        "args":order_message,
    }
    count+=1
    print(f'>>>orderId:{order_message["orderId"]},Count:{count}<<<',end='\r')
    print(order_message)
    c.prdb_mast(0).rpush('session_task_hub',message)
    break

{'appCode': '91ala', 'userId': '4509401', 'orderType': 'jiudian', 'partnerId': '5303341491', 'orderId': 'ala203403526463556'}


In [157]:
# 外卖:增加轮训订单任务
for item in c.pmdb.order_waimai.find({'id':{'$in':['bp203062425642449']}},
                                         {'id':1,'partnerId':1,'_id':0},sort=[('_id',-1)]):
    item['orderId']=item.pop('id')
#     print(item)
#     continue
    message={
        "taskName":"worker.ele.order_status_polling",
#         "taskName":"worker.ele.ele_fetch",
        "queue":"ele_tasks",
        "args":item
    }
    print(item)
    c.prdb_mast(0).rpush('session_task_hub',message)

{'partnerId': '1215426746044982493', 'orderId': 'bp203062425642449'}


In [2]:
order='''ok203361614238678
ala203359383509924
ok203313984110336
ok203312898715347
ok203305925144735
ok203290905710774
ok203267337686570
ok203250615250771
ok203232574518713
ala203212153062388
ala203144776995802
ala203141129650216
ala203403526463556'''.split('\n')
order

['ok203361614238678',
 'ala203359383509924',
 'ok203313984110336',
 'ok203312898715347',
 'ok203305925144735',
 'ok203290905710774',
 'ok203267337686570',
 'ok203250615250771',
 'ok203232574518713',
 'ala203212153062388',
 'ala203144776995802',
 'ala203141129650216',
 'ala203403526463556']

In [5]:
# 酒店:增加轮训订单任务
# order=['ala202332129231317']
for item in c.pmdb.order_jiudian.find({'id':{'$in':order}},
                                         {'id':1,'partnerId':1,'appCode':1,'_id':0},sort=[('_id',-1)]):
    item['orderId']=item.pop('id')
#     print(item)
#     continue
    message={
        "taskName":"worker.ctrip.ctrip_fetch",
        "queue":"ctrip_tasks",
        "args":item
    }
    print(item)
    c.prdb_mast(0).rpush('session_task_hub',message)

{'appCode': '91ala', 'partnerId': '5303341491', 'orderId': 'ala203403526463556'}
{'appCode': 'oklife', 'partnerId': '5283912119', 'orderId': 'ok203361614238678'}
{'appCode': '91ala', 'partnerId': '5282044858', 'orderId': 'ala203359383509924'}
{'appCode': 'oklife', 'partnerId': '5262472184', 'orderId': 'ok203313984110336'}
{'appCode': 'oklife', 'partnerId': '5261774046', 'orderId': 'ok203312898715347'}
{'appCode': 'oklife', 'partnerId': '5259711914', 'orderId': 'ok203305925144735'}
{'appCode': 'oklife', 'partnerId': '5251623791', 'orderId': 'ok203290905710774'}
{'appCode': 'oklife', 'partnerId': '5241984570', 'orderId': 'ok203267337686570'}
{'appCode': 'oklife', 'partnerId': '5236497044', 'orderId': 'ok203250615250771'}
{'appCode': 'oklife', 'partnerId': '5227717724', 'orderId': 'ok203232574518713'}
{'appCode': '91ala', 'partnerId': '5219231858', 'orderId': 'ala203212153062388'}
{'appCode': '91ala', 'partnerId': '5193417933', 'orderId': 'ala203144776995802'}
{'appCode': '91ala', 'partne

In [8]:
for item in c.pmdb.order_jiudian.find({'displayStatus':'订单异常'},
                                         {'id':1,'partnerId':1,'appCode':1,'timeline':1,'_id':0},sort=[('_id',-1)]):
    item['orderId']=item.pop('id')
#     print(item)
#     continue
    message={
        "taskName":"worker.ctrip.ctrip_fetch",
        "queue":"ctrip_tasks",
        "args":item
    }
    print(item)
#     c.prdb_mast(0).rpush('session_task_hub',message)

{'appCode': '91ala', 'partnerId': '5303341491', 'timeline': ['01-26 13:00 已下单', '01-26 13:00 待支付', '02-08 10:28 订单异常'], 'orderId': 'ala203403526463556'}
{'appCode': '91ala', 'partnerId': '5282044858', 'timeline': ['01-22 15:27 已下单', '01-22 15:27 预约成功', '01-22 15:27 等待入住', '02-08 10:28 订单异常'], 'orderId': 'ala203359383509924'}
{'appCode': '91ala', 'partnerId': '5219231858', 'timeline': ['01-09 15:26 已下单', '01-09 15:26 预约成功', '01-09 15:26 等待入住', '02-08 10:28 订单异常'], 'orderId': 'ala203212153062388'}
{'appCode': '91ala', 'partnerId': '5193417933', 'timeline': ['01-03 16:39 已下单', '01-03 16:39 预约成功', '01-03 16:39 等待入住', '02-08 10:28 订单异常'], 'orderId': 'ala203144776995802'}
{'appCode': '91ala', 'partnerId': '5191414965', 'timeline': ['01-03 08:55 已下单', '01-03 08:55 预约成功', '01-03 08:55 等待入住', '02-08 10:28 订单异常'], 'orderId': 'ala203141129650216'}
{'appCode': 'boluome', 'partnerId': '4795655063', 'timeline': ['11-03 14:47 已下单'], 'orderId': 'blm202453093749662'}
{'partnerId': '4795645147', 'timeli

In [13]:
# 出票中电影: 增加轮训订单任务
for item in c.pmdb.order_dianying.find({'displayStatus':{'$in':['订单异常','出票中']},
                                        'createdAt':{
                                            '$gte':(time.time()-3600*24*25)*1000,
                                            '$lt':(time.time()-16*60)*1000
                                        }
                                       },
                                       {'id':1,'partnerId':1,'_id':0},
                                       sort=[('_id',-1)]):
    item['orderId']=item.pop('id')
    item['partnerOrderId']=item.pop('partnerId')
#     print(item)
#     continue
    message={
        "taskName":"worker.kou.kou_fetch",
        "queue":"kou_tasks",
        "args":item
    }
    print(item)
    c.prdb_mast(0).rpush('session_task_hub',message)

{'orderId': 'ala202285885197952', 'partnerOrderId': 'a1508415920461572269'}


## 基于rabbitmq—proxy的消息publish

In [42]:
# stg
import requests
body={
    "exchangeName":"session",
    "routingKey":"task",
    "payload":{
        "taskName":"worker.yongle.order_status_polling",
        "queue":"yongle_tasks",
        "args":{
            "partnerId" : "33353024",
            "orderId" : "blm102407380811820",
            "dispatchWay" : 2
        }
    }
}
requests.post('http://192.168.2.6:8672/rabbitmq/publish',json=body).json()

{'code': 0, 'message': 'publish ok'}

In [48]:
# stg
import requests
body={
    "exchangeName":"session",
    "routingKey":"task",
    "payload":{
        "taskName":"worker.shenzhou.shenzhou_fetch",
        "queue":"shenzhou_tasks",
        "args":{
            "partnerId" : "6474805778871353356",
            "orderId" : "bl102170184925191",
            "model" : "realtime",
            "orderType":"jjj"
        }
    }
}
requests.post('http://192.168.0.6:8672/rabbitmq/publish',json=body).json()

{'code': 0, 'message': 'publish ok'}

In [72]:
# 所有品类，推送已取消状态
count=0
for order_lite in c.pmdb.order_lite_list.find({'status':8,
#                                                'id':{'$in':['ayd202759420418896']},
                                               'createdAt':{'$gte':(time.time()-3600*24)*1000,
                                                            '$lt':(time.time()-16*60)*1000}},
                                              
                                              {'id':1,'partnerId':1,'_id':0,'userId':1,'appCode':1,'orderType':1},
                                             sort=[('_id',-1)]):
    
    item=c.pmdb[f'order_{order_lite["orderType"]}'].find_one({'id':order_lite['id']},{'_id':0,'appCode':1,'id':1,'userId':1,'status':1,'displayStatus':1})
    item['orderId']=item.pop('id')
    message={
        "taskName":"worker.sync_status.push_sync_status_message",
        "queue":"sync_tasks",
        "args":[item,item.pop('appCode')]
    }
#     print(item)
    c.prdb_mast(0).rpush('session_task_hub',message)
    count+=1
    print(count,end='\r')
    time.sleep(0.1)
#     break

In [102]:
order_id_list=[item['id'] for item in c.pmdb.order_waimai.find({'displayStatus':{'$in':['已支付']},
                                          'updatedAt':{'$gte':(time.time()-3600*24*21)*1000,
                                                      '$lt':(time.time()-360)*1000},
                                         },
                                         {'id':1,'partnerId':1,'_id':0},sort=[('_id',-1)])]


In [28]:
# bl102668270191062
# bl102668524732908
# bl202753949076886 
order_id_list=['bl102668270191062','bl102668524732908']

In [29]:
# 订单置为等待退款
count = 0
# mdb,rdb = c.smdb,c.srdb_new(1) # stg
mdb,rdb = c.pmdb,c.prdb_paym(1) # pro
for order_lite in mdb.order_lite_list.find({'id':{'$in':order_id_list}},
                                           {'id':1,'orderType':1}):
#     {
#                     "orderId": "vjr202804331326438",
#                     "paymentSerial": "vjr212804331326439",
#                     "orderType": "huafei",
#                     "appCode": "vipjinrong"
#                 }
    item=mdb[f'order_{order_lite["orderType"]}'].find_one({'id':order_lite['id']},{'_id':0,'paymentSerial':1,'appCode':1,'id':1,'orderType':1})
                
    item['orderId']=item.pop('id')
    print(item['orderId'])
    rdb.rpush('compensation',item)

bl102668270191062
bl102668524732908


In [56]:
# 1. channel 改成ofpay （payment表，mongo表）
# 2. 高阳worker停掉，不要置为异常
# 3. 遍历今日包商加油卡异常订单

import requests
url='http://sessionapi.localdomain:8010/sm/order/update'
for order in order_message:
    order_id=order['orderId']
    partner_id=order['partnerId']
    body={'id':order_id,'channel':'ofPay','partnerId':partner_id}
    resp=requests.post(url=url,json=body).json()
    if resp['code']!=0:
        print(resp)
        raise KeyError
    c.prdb_paym(1).hset(f"p:bsb:{resp['data']['paymentSerial']}","supplier","ofPay")

In [57]:
order_message = [{'orderId':item.split('\t')[0],'partnerId':item.split('\t')[-1],'price':item.split('\t')[2],'displayStatus':item.split('\t')[-2]} for item in message.split('\n')]
[{"orderId":item['orderId'],"orderType":"jiayouka"} for item in order_message if item['displayStatus']=='成功']

[{'orderId': 'bsb203618400388943', 'orderType': 'jiayouka'},
 {'orderId': 'bsb203618421607480', 'orderType': 'jiayouka'},
 {'orderId': 'bsb203618422556390', 'orderType': 'jiayouka'},
 {'orderId': 'bsb203618426095336', 'orderType': 'jiayouka'},
 {'orderId': 'bsb203618430520391', 'orderType': 'jiayouka'},
 {'orderId': 'bsb203618433535049', 'orderType': 'jiayouka'},
 {'orderId': 'bsb203618433697526', 'orderType': 'jiayouka'},
 {'orderId': 'bsb203618435139324', 'orderType': 'jiayouka'},
 {'orderId': 'bsb203618435663613', 'orderType': 'jiayouka'},
 {'orderId': 'bsb203618439433296', 'orderType': 'jiayouka'},
 {'orderId': 'bsb203618440251138', 'orderType': 'jiayouka'},
 {'orderId': 'bsb203618450458512', 'orderType': 'jiayouka'},
 {'orderId': 'bsb203618452409442', 'orderType': 'jiayouka'},
 {'orderId': 'bsb203618452933731', 'orderType': 'jiayouka'},
 {'orderId': 'bsb203618457290523', 'orderType': 'jiayouka'},
 {'orderId': 'bsb203618464499489', 'orderType': 'jiayouka'},
 {'orderId': 'bsb2036184

In [49]:
message = """bsb203618400388943	9030160003142813	100	成功	S1802145255959
bsb203618421607480	9030160003080816	100	成功	S1802145265655
bsb203618422556390	9030160002367846	100	成功	S1802145266776
bsb203618426095336	9000000002349904	100	成功	S1802145267835
bsb203618430520391	9030160003151852	100	成功	S1802145268696
bsb203618433535049	9030160003497241	100	成功	S1802145270383
bsb203618433697526	9030160003444696	100	成功	S1802145271055
bsb203618435139324	9030160003253025	100	成功	S1802145273213
bsb203618435663613	9030160002291625	100	成功	S1802145274163
bsb203618439433296	9030160002291625	100	成功	S1802145275748
bsb203618440251138	9030160002359163	100	成功	S1802145276664
bsb203618450458512	9030160002463965	200	成功	S1802145279041
bsb203618452409442	9099990000612153	100	成功	S1802145280488
bsb203618452933731	9030090002943213	100	成功	S1802145281124
bsb203618457290523	9030160002531142	100	成功	S1802145283465
bsb203618464499489	9030160003126823	100	成功	S1802145285829
bsb203618473134000	9030160002894235	100	成功	S1802145287802
bsb203618474313649	9030160002888430	100	成功	S1802145288732
bsb203618478245820	9030160003437030	100	成功	S1802145290005
bsb203618480490296	9030160002556742	100	成功	S1802145292853
bsb203618487420875	9030160002881666	100	成功	S1802145293664
bsb203618487699265	9030160002686454	100	成功	S1802145294526
bsb203618491500360	9030160002560302	100	成功	S1802145297997
bsb203618497644514	9030160003237774	200	成功	S1802145301789
bsb203618499479525	9030160001602139	100	成功	S1802145302701
bsb203618501183463	9000000002481055	100	成功	S1802145303342
bsb203618504067053	9030160002910909	100	成功	S1802145305434
bsb203618504984559	9030160002314379	100	成功	S1802145305944
bsb203618506017951	9030160003273373	100	成功	S1802145307039
bsb203618506573655	9030160003080816	100	成功	S1802145308248
bsb203618507212787	9030160003304182	100	成功	S1802145310871
bsb203618510736559	9030160002894236	100	成功	S1802145312602
bsb203618515093356	9030160002391293	100	成功	S1802145323171
bsb203618520336242	9030160002881666	100	成功	S1802145324195
bsb203618522810377	9030160003229719	100	成功	S1802145326747
bsb203618534591691	9030160002556742	100	成功	S1802145328418
bsb203618536458119	9030160003368402	100	成功	S1802145329840
bsb203618537375625	9030160002291625	100	成功	S1802145331269
bsb203618541684760	9030160003399360	100	成功	S1802145333305
bsb203618543766744	9030160002661660	100	成功	S1802145334310
bsb203618545108882	9030160002455811	100	成功	S1802145335145
bsb203618547845149	9030090000245272	100	成功	S1802145335941
bsb203618549009632	9030160002196585	100	成功	S1802145337140
bsb203618551400348	9030010003648389	100	成功	S1802145338193
bsb203618555332516	9030160002686454	100	成功	S1802145338866
bsb203618555840552	9030160002213305	500	成功	S1802145339884
bsb203618565015604	9030160001816327	100	成功	S1802145344082
bsb203618569063672	9030160002531224	100	成功	S1802145345209
bsb203618570898682	9030160002567235	100	成功	S1802145348009
bsb203618579680516	9030160003135877	100	成功	S1802145348844
bsb203618586904649	9030160002291625	100	成功	S1802145350954
bsb203618588724495	9030160003226211	100	成功	S1802145351894
bsb203618829666480	9030160002221322	100	成功	S1802145353563
bsb203618836875443	9030160003136378	100	成功	S1802145355350
bsb203618846181562	9030160002555026	100	成功	S1802145356711
bsb203618850506944	9030160003395957	100	成功	S1802145362342
bsb203618851031233	9030160001373093	100	成功	S1802145363151
bsb203618854407699	9030160002678642	100	成功	S1802145365706
bsb203618857322701	9030160002071600	100	成功	S1802145366328
bsb203618860321100	9030160003479203	100	成功	S1802145366983
bsb203618866890967	9030160002173739	100	成功	S1802145367646
bsb203618881554776	9030160002486344	100	成功	S1802145368530
bsb203618886273371	9030160003274142	100	成功	S1802145369058
bsb203618896496996	9030160002253991	100	成功	S1802145369693
bsb203618897021285	9030160002276517	100	成功	S1802145370482
bsb203618898579004	9030160002431652	100	成功	S1802145373992
bsb203618902904384	9030160003299714	100	成功	S1802145374870
bsb203618906982763	9030160002274771	100	成功	S1802145375387
bsb203618924431627	9030160003299714	100	成功	S1802145376226
bsb203618927283795	9030160003471297	100	成功	S1802145377406
bsb203618932951313	9030160003241223	100	成功	S1802145378016
bsb203618933197186	9030160003136378	100	成功	S1802145378582
bsb203618933968474	9030160003105020	200	成功	S1802145379332
bsb203618936720990	9030160003229719	100	成功	S1802145381514
bsb203618937653640	9030160002431652	100	成功	S1802145383293
bsb203618938949219	9030160003125770	100	成功	S1802145383983
bsb203618943420814	9030160003055587	100	成功	S1802145384742
bsb203618949056915	9030160002861899	100	成功	S1802145385342
bsb203618952743214	9030160003390568	100	成功	S1802145386000
bsb203618954824091	9030090000431234	100	成功	S1802145386709
bsb203618954940024	9030160002689618	100	成功	S1802145387219
bsb203618974239042	9030160003136378	100	成功	S1802145388237
bsb203618975942979	9030160001342840	100	成功	S1802145388834
bsb203618995310232	9030160002449553	100	成功	S1802145389675
bsb203619006876003	9030160001782368	100	成功	S1802145390396
bsb203619014068699	9030160000980980	100	成功	S1802145391215
bsb203619017214431	9030160003045353	100	成功	S1802145391814
bsb203619019951797	9030010002914678	100	成功	S1802145392596
bsb203619021015525	9030160002783004	100	成功	S1802145399372
bsb203619021655737	9030160003136378	100	成功	S1802145400019
bsb203619030583786	9030160002062018	200	充值中	S1802145400864
bsb203619031108076	9030160003399880	100	成功	S1802145401903
bsb203619040530124	9030160003100567	100	成功	S1802145402582
bsb203619041610107	9030160003304182	100	成功	S1802145403125
bsb203619050654081	9030160003172230	100	成功	S1802145404096"""

In [59]:
for lite in order_message:
    item = c.pmdb['order_jiayouka'].find_one({"id":lite['orderId']})
    order_id = item['id']
    display_status = item['displayStatus']
    print(order_id,display_status)

bsb203618400388943 已完成
bsb203618421607480 已完成
bsb203618422556390 已完成
bsb203618426095336 已完成
bsb203618430520391 已完成
bsb203618433535049 已完成
bsb203618433697526 已完成
bsb203618435139324 已完成
bsb203618435663613 已完成
bsb203618439433296 已完成
bsb203618440251138 已完成
bsb203618450458512 已完成
bsb203618452409442 已完成
bsb203618452933731 已完成
bsb203618457290523 已完成
bsb203618464499489 已完成
bsb203618473134000 已完成
bsb203618474313649 已完成
bsb203618478245820 已完成
bsb203618480490296 已完成
bsb203618487420875 已完成
bsb203618487699265 已完成
bsb203618491500360 已完成
bsb203618497644514 已完成
bsb203618499479525 已完成
bsb203618501183463 已完成
bsb203618504067053 已完成
bsb203618504984559 已完成
bsb203618506017951 已完成
bsb203618506573655 已完成
bsb203618507212787 已完成
bsb203618510736559 已完成
bsb203618515093356 已完成
bsb203618520336242 已完成
bsb203618522810377 已完成
bsb203618534591691 已完成
bsb203618536458119 已完成
bsb203618537375625 已完成
bsb203618541684760 已完成
bsb203618543766744 已完成
bsb203618545108882 已完成
bsb203618547845149 已完成
bsb203618549009632 已完成
bsb20361855

In [47]:
# 3. 遍历今日包商加油卡异常订单
start = time.mktime((2018,2,14,0,0,0,0,0,0))*1000
for lite in c.pmdb.order_lite_list.find({"appCode":"bsb",'createdAt':{"$gte":start}}):
    item = c.pmdb[f'order_{lite["orderType"]}'].find_one({"id":lite['id']})
    order_id = item['id']
    display_status = item['displayStatus']
    print(order_id,display_status)

bsb203619526560623 充值中
bsb203619520531304 充值中
bsb203619512027890 待支付
bsb203619508064314 充值中
bsb203619492629220 充值中
bsb203619490663138 待支付
bsb203619485682399 待支付
bsb203619483044687 充值中
bsb203619480554317 待支付
bsb203619473985572 待支付
bsb203619470577698 待支付
bsb203619468495683 充值中
bsb203619464186576 待支付
bsb203619463106585 充值中
bsb203619457878843 待支付
bsb203619457092410 充值中
bsb203619455504405 充值中
bsb203619449999378 待支付
bsb203619449212943 充值中
bsb203619447540421 充值中
bsb203619446984715 充值中
bsb203619445936137 充值中
bsb203619444494342 充值中
bsb203619439151807 充值中
bsb203619438086961 充值中
bsb203619437940737 充值中
bsb203619436907312 充值中
bsb203619434023725 已取消
bsb203619431549618 充值中
bsb203619430207479 待支付
bsb203619426962097 已取消
bsb203619425504039 待支付
bsb203619423784948 充值中
bsb203619420408491 充值中
bsb203619417524906 充值中
bsb203619408973804 充值中
bsb203619403483927 已取消
bsb203619401517845 已取消
bsb203619401009823 已取消
bsb203619400747678 充值中
bsb203619398225893 充值中
bsb203619396029082 充值中
bsb203619395357451 充值中
bsb20361939

bsb203618679195694 已取消
bsb203618678393008 订单异常
bsb203618678016043 订单异常
bsb203618677868719 已取消
bsb203618677753897 已取消
bsb203618676673905 订单异常
bsb203618676557997 已取消
bsb203618674985131 订单异常
bsb203618672379936 订单异常
bsb203618671037804 已取消
bsb203618669758494 订单异常
bsb203618667761002 已取消
bsb203618665532777 订单异常
bsb203618662911334 已取消
bsb203618661484701 已取消
bsb203618659797014 已取消
bsb203618658585951 已取消
bsb203618656782352 订单异常
bsb203618656651279 订单异常
bsb203618656357724 已取消
bsb203618655209484 已取消
bsb203618654423050 已取消
bsb203618653080921 订单异常
bsb203618651261069 已取消
bsb203618651146246 订单异常
bsb203618649950346 已取消
bsb203618648655876 已取消
bsb203618647984263 已取消
bsb203618647706961 订单异常
bsb203618647066758 已取消
bsb203618644576389 订单异常
bsb203618643527812 已取消
bsb203618641168512 订单异常
bsb203618641037438 订单异常
bsb203618640760134 已取消
bsb203618639726716 已取消
bsb203618637221188 已取消
bsb203618636990454 订单异常
bsb203618631862388 已取消
bsb203618631060797 订单异常
bsb203618629240944 已取消
bsb203618628570426 订单异常
bsb2036186238669

bsb203617616693538 已取消
bsb203617616562465 已完成
bsb203617616300320 等待退款
bsb203617616069573 已完成
bsb203617613956170 等待退款
bsb203617611596869 已取消
bsb203617610826686 已完成
bsb203617610402069 已完成
bsb203617608042770 已完成
bsb203617607649553 已完成
bsb203617606600976 等待退款
bsb203617605945614 已取消
bsb203617605190581 已完成
bsb203617604797362 等待退款
bsb203617604142000 已取消
bsb203617603486638 已取消
bsb203617603470399 已完成
bsb203617603224493 已取消
bsb203617602159677 已完成
bsb203617601489161 等待退款
bsb203617600586812 已取消
bsb203617600309510 已取消
bsb203617599276091 已完成
bsb203617598605571 已完成
bsb203617598489658 已取消
bsb203617598358585 已取消
bsb203617597588392 已完成
bsb203617597556989 已完成
bsb203617596670886 已取消
bsb203617596670885 已完成
bsb203617595328764 已完成
bsb203617595197691 已完成
bsb203617594442656 已取消
bsb203617594049439 已完成
bsb203617593918366 已取消
bsb203617592445178 已完成
bsb203617592314105 已取消
bsb203617591018538 已取消
bsb203617591018537 已完成
bsb203617589692660 已取消
bsb203617589068691 已完成
bsb203617589037299 已完成
bsb203617588675473 已完成
bsb203

bsb203617408303844 已完成
bsb203617408041699 已完成
bsb203617407895432 已完成
bsb203617406060422 已取消
bsb203617405960789 已取消
bsb203617405944544 已取消
bsb203617405027039 已完成
bsb203617404371678 已取消
bsb203617403570049 已完成
bsb203617403454172 已取消
bsb203617403339348 已完成
bsb203617402929882 充值中
bsb203617402290769 已完成
bsb203617402274520 已取消
bsb203617402128250 已取消
bsb203617401225942 已取消
bsb203617400948601 已完成
bsb203617400570580 已完成
bsb203617400570579 已完成
bsb203617400455757 已取消
bsb203617399113592 已取消
bsb203617398882891 已取消
bsb203617398604496 已完成
bsb203617398096457 已取消
bsb203617397818063 已取消
bsb203617397540725 已完成
bsb203617396754292 已完成
bsb203617396623219 已取消
bsb203617396523587 已完成
bsb203617395851979 已取消
bsb203617395836784 已取消
bsb203617395458761 已完成
bsb203617394819649 已完成
bsb203617394033216 已取消
bsb203617393902143 已完成
bsb203617393623748 已完成
bsb203617393346411 已完成
bsb203617392691049 已取消
bsb203617392166760 已完成
bsb203617392035687 已取消
bsb203617391542843 已完成
bsb203617389938532 已取消
bsb203617388134969 已取消
bsb20361738

bsb203617292189995 已完成
bsb203617292058921 已取消
bsb203617292042674 已完成
bsb203617292027489 已取消
bsb203617292058920 已取消
bsb203617292042673 已取消
bsb203617291796775 已取消
bsb203617291649455 已取消
bsb203617291503197 已取消
bsb203617291503196 已完成
bsb203617291256237 已完成
bsb203617291125163 已完成
bsb203617290978906 已取消
bsb203617290847833 已完成
bsb203617290617122 已完成
bsb203617290454616 已取消
bsb203617290469801 已取消
bsb203617290486048 已取消
bsb203617290338726 已完成
bsb203617290192470 已取消
bsb203617290192469 已完成
bsb203617289945508 已取消
bsb203617289961756 等待退款
bsb203617289961754 已完成
bsb203617289961753 已取消
bsb203617289814435 已取消
bsb203617289537106 等待退款
bsb203617289421217 已完成
bsb203617289306392 已完成
bsb203617289306390 已完成
bsb203617289159071 已取消
bsb203617289159070 已完成
bsb203617289044243 已取消
bsb203617289044242 已完成
bsb203617288913169 已取消
bsb203617288913168 已完成
bsb203617288881739 已取消
bsb203617288750664 已取消
bsb203617288765851 已取消
bsb203617288651022 已完成
bsb203617288651021 已取消
bsb203617288619591 已取消
bsb203617288651020 已完成
bsb203617

bsb203617277347164 已取消
bsb203617277247540 已取消
bsb203617277247539 已完成
bsb203617277247537 已完成
bsb203617277231289 已完成
bsb203617277231288 已完成
bsb203617277247534 已完成
bsb203617277247532 已完成
bsb203617277216091 已完成
bsb203617277231284 已完成
bsb203617277231283 已完成
bsb203617277247530 已完成
bsb203617277231281 已完成
bsb203617277216089 已完成
bsb203617277247528 已取消
bsb203617277216088 已取消
bsb203617277216086 已取消
bsb203617277247527 已取消
bsb203617277100206 已取消
bsb203617277100205 已完成
bsb203617277116454 已取消
bsb203617277116453 已完成
bsb203617277100202 已完成
bsb203617277100201 已完成
bsb203617277100198 等待退款
bsb203617277116451 已完成
bsb203617277100197 已完成
bsb203617277100196 已取消
bsb203617277100194 已取消
bsb203617277085008 已完成
bsb203617276953934 已取消
bsb203617276969120 已完成
bsb203617276969119 已完成
bsb203617276953932 已完成
bsb203617276969118 等待退款
bsb203617276969117 已完成
bsb203617276969116 已完成
bsb203617276953931 已完成
bsb203617276985369 已完成
bsb203617276953929 已完成
bsb203617276985367 已完成
bsb203617276969113 已完成
bsb203617276969112 已完成
bsb203617

bsb203617252868008 已取消
bsb203617251688359 已取消
bsb203617251263590 已取消
bsb203617251180367 已取消
bsb203617251032997 已取消
bsb203617250608228 已取消
bsb203617249428579 已取消
bsb203617248804772 已取消
bsb203617248427852 已取消
bsb203617248296779 已取消
bsb203617247724641 已取消
bsb203617247069279 已取消
bsb203617246723912 已取消
bsb203617245675335 已取消
bsb203617244872606 已取消
bsb203617244888901 已取消
bsb203617244447835 已取消
bsb203617240039233 已取消
bsb203617238549592 已取消
bsb203617237632087 已取消
bsb203617236845654 已取消
bsb203617236238137 已取消
bsb203617235451704 已取消
bsb203617235042202 已取消
bsb203617225211798 已取消
bsb203617224425364 已取消
bsb203617223869519 已取消
bsb203617222606644 已取消
bsb203617221672850 已取消
bsb203617220592717 已取消
bsb203617219837840 已取消
bsb203617216839471 已取消
bsb203617210154796 已取消
bsb203617207254918 已取消
bsb203617204125479 已取消
bsb203617203207973 已取消
bsb203617202373701 已取消
bsb203617201241891 已取消
bsb203617200669764 已取消
bsb203617196737599 已取消
bsb203617196654366 已取消
bsb203617194409852 已取消
bsb203617193623419 已取消
bsb20361718

In [64]:
# 包商现在所有等待退款的，需要流水号和退款金额和下单时间
import tablib
dataset = tablib.Dataset(headers=['订单号','下单时间','流水号','退款金额'])
start = time.mktime((2018,2,13,0,0,0,0,0,0))*1000
for lite in c.pmdb.order_lite_list.find({"appCode":"bsb","status":11,'createdAt':{"$gte":start}}):
    item = c.pmdb[f'order_{lite["orderType"]}'].find_one({"id":lite['id']})
    order_id = item['id']
    url = "http://dpp.boluome.com/order/search/{}".format(order_id)
    resp = requests.get(url).json()
    refund_id = resp['data']['refundFlowId']
    payment_id = resp['data']['paymentSerial']
    refund_price = float(c.prdb_paym(2).hget(f'r:{item["appCode"]}:{refund_id}','price'))/100
    created_at = time.strftime("%Y-%m-%dT%H:%M:%S",time.localtime(item['createdAt']/1000))
    dataset.append([order_id,created_at,payment_id,refund_price])
#     print(order_id,created_at,payment_id,refund_id,refund_price,item['price'])
with open("包商等待退款订单信息.xlsx","wb") as f:
    f.write(dataset.xlsx)

/usr/local/python3/lib/python3.6/site-packages/openpyxl/worksheet/worksheet.py:303: UserWarning: Using a coordinate with ws.cell is deprecated. Use ws[coordinate] instead
  warn("Using a coordinate with ws.cell is deprecated. Use ws[coordinate] instead")


In [63]:
for item in c.pmdb['order_jiayouka'].find({"displayStatus":"订单异常",'createdAt':{"$gte":start}}):
    print(item['timeline_new'][-2])